<a href="https://colab.research.google.com/github/isharaWijayaratne/TimeSeriesModelSelectionChatbots/blob/main/FFORMS_LLM_yearly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai pinecone-client[grpc] datasets tiktoken langchain pandas gradio

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import openai

#def get_embedding(text, model="text-embedding-ada-002"): # model = "deployment_name"
#    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00


In [ ]:
#from openai.embeddings_utils import get_embedding

from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Pinecone
from langchain.agents import Tool
from langchain.agents import initialize_agent

from tqdm.auto import tqdm # provides a progress bar
from uuid import uuid4 #UUID (Universally Unique Identifier)

import pinecone
import gradio as gr

In [ ]:
df = pd.read_csv('TRAINING_yearly_1000_adjusted.csv')

In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 3.6 MB/s eta 0:00:00


In [ ]:

from langchain_openai import OpenAIEmbeddings

openai_api_key = 'OPENAI API KEY'
model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=openai_api_key,
)

In [ ]:
!pip install pinecone-client

from pinecone import Pinecone

pc = Pinecone(api_key="PINECONE API KEY")
index = pc.Index("businesschatbot2", dimension=1536, metric='cosine')
index_name = "businesschatbot2"


In [ ]:
#NEED ONLY TO RUN ONCE
batch_size = 5

texts = []
metadatas = []

for i in tqdm(range(0, len(df), batch_size)):
    i_end = min(len(df), i+batch_size)
    batch = df.iloc[i:i_end]

    metadatas = [{
        'text': str(record['Features']),
        'text': str(record['classlabels'])
    } for j, record in batch.iterrows()]
    documents = batch['classlabels']
    batch['classlabels'] = batch['classlabels'].fillna('')
    documents = batch['classlabels'].astype(str)
    embeds = embed.embed_documents(documents)
    ids = batch['Feature_ID'].astype(str)
    index.upsert(vectors=zip(ids, embeds, metadatas))

  0%|          | 0/2000 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fillna('')
<ipython-input-21-e31c2fda82fb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fillna('')
<ipython-input-21-e31c2fda82fb>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['classlabels'] = batch['classlabels'].fi

In [ ]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 10000}},
 'total_vector_count': 10000}

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
!pip install --upgrade --quiet  langchain-pinecone

In [ ]:
from langchain_pinecone import PineconeVectorStore

In [ ]:
text_field = "text"

vectorstore = PineconeVectorStore(
    index, embed, text_field
)

In [ ]:
from sqlalchemy.sql.expression import true
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)


In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


In [ ]:

tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'based on the most similar row of the classlabels column'
            'answer with more information'
        )
    )
]

In [ ]:
agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=False,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
with gr.Blocks(theme=gr.themes.Soft()) as demo: #"gradio/seafoam"
  # Adding a title using Markdown
  gr.Markdown("# FFORMS Yearly GPT Chatbot")

  chatbot = gr.Chatbot()
  msg = gr.components.Textbox(label="timeseries_features")
  clear = gr.ClearButton([msg, chatbot])

  def respond(message, chat_history):
    bot_message = agent.run(message)
    print(bot_message)
    chat_history.append((message, bot_message))
    return "", chat_history

  msg.submit(respond, [msg, chatbot], [msg, chatbot])

demo.launch(share=True, debug=true)